## Final Project Submission

Please fill out:
* Student name: Victor Kigen, Bosco, 
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Loading movie gross data from Box Office Mojo
bom_movie_gross_data = pd.read_csv('zippedData/bom.movie_gross.csv', encoding='ISO-8859-1', low_memory=False)

# Loading Rotten Tomatoes movie info data with tab-separated values
info_data = pd.read_csv('zippedData/rt.movie_info.tsv', delimiter='\t', encoding='ISO-8859-1', low_memory=False)

# Loading Rotten Tomatoes review data, also tab-separated
review_data = pd.read_csv('zippedData/rt.reviews.tsv', delimiter='\t', encoding='ISO-8859-1', low_memory=False)

# Loading TMDb (The Movie Database) movie data
tmdb_movie_data = pd.read_csv('zippedData/tmdb.movies.csv', encoding='ISO-8859-1', low_memory=False)

# Loading movie budget data from The Numbers database
budget_data = pd.read_csv('zippedData/tn.movie_budgets.csv', encoding='ISO-8859-1', low_memory=False)


In [5]:
bom_movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [6]:
bom_movie_gross_data[bom_movie_gross_data[['foreign_gross']].isnull().any(axis=1)]
bom_movie_gross_data['foreign_gross'] = pd.to_numeric(bom_movie_gross_data['foreign_gross'].replace('[\$,]', '', regex=True))
foreign_gross_median = bom_movie_gross_data['foreign_gross'].median() 

bom_movie_gross_data['foreign_gross'].fillna(foreign_gross_median, inplace=True)

In [7]:
bom_movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   3387 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


In [8]:
bom_movie_gross_data[bom_movie_gross_data[['domestic_gross']].isnull().any(axis=1)]
bom_movie_gross_data['domestic_gross'] = pd.to_numeric(bom_movie_gross_data['domestic_gross'].replace('[\$,]', '', regex=True))
domestic_gross_median = bom_movie_gross_data['domestic_gross'].median() 

bom_movie_gross_data['domestic_gross'].fillna(domestic_gross_median, inplace=True)
bom_movie_gross_data.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010


In [9]:
mode_value = bom_movie_gross_data['studio'].mode()[0]
bom_movie_gross_data['studio'].fillna(mode_value, inplace=True)


In [10]:
bom_movie_gross_data['studio'] = bom_movie_gross_data['studio'].str.upper()

In [11]:
bom_movie_gross_data['year'].value_counts()

2015    450
2016    436
2012    400
2011    399
2014    395
2013    350
2010    328
2017    321
2018    308
Name: year, dtype: int64

In [12]:
columns_to_drop = ['synopsis', 'dvd_date', 'studio', 'currency', 'box_office']
info_data = info_data.drop(columns=columns_to_drop)

info_data.head()


,id,rating,genre,director,writer,theater_date,runtime
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971",104 minutes
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012",108 minutes
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996",116 minutes
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994",128 minutes
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,200 minutes


In [13]:
info_data[['director', 'writer']] = info_data[['director', 'writer']].fillna('N/A')

info_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   rating        1557 non-null   object
 2   genre         1552 non-null   object
 3   director      1560 non-null   object
 4   writer        1560 non-null   object
 5   theater_date  1201 non-null   object
 6   runtime       1530 non-null   object
dtypes: int64(1), object(6)
memory usage: 85.4+ KB


In [14]:
# Convert the date strings to datetime objects

info_data['theater_date'] = pd.to_datetime(info_data['theater_date'], errors='coerce')


date_mean = info_data['theater_date'].mean()
info_data['theater_date'].fillna(date_mean, inplace=True)

# Format the datetime objects to the desired string format
info_data['theater_date'] = info_data['theater_date'].dt.strftime('%Y-%m-%d')


In [15]:
info_data['runtime'] = pd.to_numeric(info_data['runtime'].replace(' minutes', '', regex=True))
info_data.head()

runtime_mean = info_data['runtime'].mean()
info_data['runtime'].fillna(runtime_mean, inplace=True)

mode_value = info_data['rating'].mode()[0]
info_data['rating'].fillna(mode_value, inplace=True)
info_data.head()

mode_value = info_data['genre'].mode()[0]
info_data['genre'].fillna(mode_value, inplace=True)
info_data.head()


,id,rating,genre,director,writer,theater_date,runtime
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,1971-10-09,104.0
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,108.0
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,1996-09-13,116.0
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,1994-12-09,128.0
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,1992-03-03,200.0


In [16]:
review_data.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [17]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [18]:
review_data['rating'].unique()



array(['3/5', nan, 'C', '2/5', 'B-', '2/4', 'B', '3/4', '4/5', '4/4',
       '6/10', '1/4', '8', '2.5/4', '4/10', '2.0/5', '3/10', '7/10', 'A-',
       '5/5', 'F', '3.5/4', 'D+', '1.5/4', '3.5/5', '8/10', 'B+', '9/10',
       '2.5/5', '7.5/10', '5.5/10', 'C-', '1.5/5', '1/5', '5/10', 'C+',
       '0/5', '6', '0.5/4', 'D', '3.1/5', '3/6', '4.5/5', '0/4', '2/10',
       'D-', '7', '1/10', '3', 'A+', 'A', '4.0/4', '9.5/10', '2.5',
       '2.1/2', '6.5/10', '3.7/5', '8.4/10', '9', '1', '7.2/10', '2.2/5',
       '0.5/10', '5', '0', '2', '4.5', '7.7', '5.0/5', '8.5/10', '3.0/5',
       '0.5/5', '1.5/10', '3.0/4', '2.3/10', '4.5/10', '4/6', '3.5',
       '8.6/10', '6/8', '2.0/4', '2.7', '4.2/10', '5.8', '4', '7.1/10',
       '5/4', 'N', '3.5/10', '5.8/10', 'R', '4.0/5', '0/10', '5.0/10',
       '5.9/10', '2.4/5', '1.9/5', '4.9', '7.4/10', '1.5', '2.3/4',
       '8.8/10', '4.0/10', '2.2', '3.8/10', '6.8/10', '7.3', '7.0/10',
       '3.2', '4.2', '8.4', '5.5/5', '6.3/10', '7.6/10', '8.1/10',
  

In [19]:
for index, value in review_data['rating'].iteritems():
    if isinstance(value, str): 
        if '/' in value: 
            if ' ' in value: 
                whole, fraction = value.split(' ')
                numerator, denominator = fraction.split('/')
                new_rating = float(whole) + (float(numerator) / float(denominator))
                review_data.at[index, 'rating'] = round(new_rating, 2)

            else:  # Regular fraction
                numerator, denominator = value.split('/')
                new_rating = float(numerator) / float(denominator)
                review_data.at[index, 'rating'] = round(new_rating, 2)



In [20]:
review_data['top_critic'].unique()

review_data['date'] = pd.to_datetime(review_data['date'], errors='coerce')
review_data['date'] = review_data['date'].dt.strftime('%Y-%m-%d')

In [21]:
columns_to_drop = ['critic', 'fresh', 'publisher']
review_data = review_data.drop(columns=columns_to_drop)


In [33]:
tmdb_movie_data.head()


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [34]:
tmdb_movie_data['popularity'].unique()

array([33.533, 28.734, 28.515, ...,  0.612,  0.609,  0.607])